In [1]:
import numpy
import os
import pandas
import pickle
import tensorflow as tf

import lightgbm as lgb

from IPython.display import clear_output
from itertools import product
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.optimizers import Adam

In [9]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
def generate_large_combinations(array):

    grid_indices = [i for i in range(9) if i != 4]

    # Generate all combinations of 0 (remove) and 1 (keep) for the grid elements except the center
    for combination in product([0, 1], repeat=len(grid_indices)):
        # Create a new array copy to modify
        new_array = numpy.copy(array)

        new_array = new_array.reshape(48, 48, 4)
        
        # Apply the combination to the grid
        for i, keep in zip(grid_indices, combination):
            if not keep:
                new_array[(16*i):(16*(i+1)), (16*i):(16*(i+1)), :] = 255, 255, 255, 255
        
        new_array = new_array.reshape((48, 48, 4))

        yield new_array

In [3]:
def generate_train_test_data(path : Path):
    files = os.listdir(str(path))

    images = [file for file in files if file.endswith(".png")]

    x_arrays = []
    y_arrays = []

    for image_name in images:
        image_path = path / image_name

        image = Image.open(str(image_path))
        image = image.convert("RGBA")

        data = numpy.array(image)

        train_y = data[16:32, 16:32, :].copy()
        train_y = train_y.reshape(-1, 16, 16, 4)

        data[16:32, 16:32, :] = 255, 255, 255, 255

        data = generate_large_combinations(data)

        unpacked_data = list(data)

        x_array = numpy.array(unpacked_data)
        y_array = numpy.tile(train_y, (len(unpacked_data), 1, 1, 1))

        x_arrays.append(x_array)
        y_arrays.append(y_array)

    return numpy.concatenate(x_arrays, axis=0), numpy.concatenate(y_arrays, axis=0)

In [4]:
x, y = generate_train_test_data(Path.cwd() / "Presets" / "Road")

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.01, random_state=10101)

In [5]:
model = models.Sequential([
    layers.Input(shape=(48, 48, 4)),

    layers.Conv2D(32, (16, 16), activation="relu", padding="same"),
    layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
    layers.MaxPooling2D((3, 3)),
    layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
    
    layers.Conv2D(4, (3, 3), activation="relu", padding="same"),
    
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),

    layers.Dense(4, activation="linear")
])

model.compile(optimizer=Adam(learning_rate=0.001), loss="mse", metrics=["mae"])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │        32,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 4)      │         4,612 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16, 16, 128)    │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16, 16, 64)     │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16, 16, 4)      │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 92,808 (362.53 KB)

 Trainable params: 92,808 (362.53 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
i = 3

single_x_test = X_test[i].copy()
single_y_test = y_test[i].copy()

single_x_test = single_x_test.reshape((-1, 48, 48, 4))

predictions = model.predict(single_x_test)

predictions = numpy.clip(predictions, 0, 255)
predictions = numpy.where(predictions < 128, 0, 255)
predictions = predictions.astype(numpy.uint8)

combined_image = single_x_test.copy()[0]
combined_image[16:32, 16:32, :] = predictions[0]

Image.fromarray(single_x_test[0]).show()
Image.fromarray(predictions[0]).show()
Image.fromarray(combined_image).show()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


In [7]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    verbose=1
)

with open("model_decent.pckl", 'wb') as file:
    pickle.dump(model, file)

# with open("model_decent.pckl", "rb") as file:
#     model = pickle.load(file)

Epoch 1/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - loss: 10969.7168 - mae: 77.4553 - val_loss: 7602.3418 - val_mae: 60.7970
Epoch 2/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - loss: 7500.5386 - mae: 62.3950 - val_loss: 7221.0859 - val_mae: 58.8397
Epoch 3/100
393/393 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - loss: 6866.4575 - mae: 59.1660 - val_loss: 5292.2808 - val_mae: 48.5383
Epoch 4/100
  1/393 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - loss: 6589.8794 - mae: 55.0464

KeyboardInterrupt: 

In [ ]:
i = 3

single_x_test = X_test[i].copy()
single_y_test = y_test[i].copy()

single_x_test = single_x_test.reshape((-1, 48, 48, 4))

predictions = model.predict(single_x_test)

predictions = numpy.clip(predictions, 0, 255)
predictions = numpy.where(predictions < 128, 0, 255)
predictions = predictions.astype(numpy.uint8)

combined_image = single_x_test.copy()[0]
combined_image[16:32, 16:32, :] = predictions[0]

Image.fromarray(single_x_test[0]).show()
Image.fromarray(predictions[0]).show()
Image.fromarray(combined_image).show()



In [ ]:
def spiral_traverse_and_capture(tile_size, n, model):
    if tile_size % 2 == 0:
        mid_x, mid_y = (tile_size // 2) - 1, (tile_size // 2) - 1
    else:
        mid_x, mid_y = tile_size // 2, tile_size // 2

    board = numpy.full((tile_size * n, tile_size * n, 4), (255, 255, 255, 255), dtype=numpy.uint8)
    
    grid = numpy.zeros((tile_size, tile_size), dtype=int)
    
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    current_dir = 0
    
    steps = 1
    steps_taken = 0
    changes = 0
    
    x, y = mid_x, mid_y
    grid[x][y] = 1
    
    images = []
    
    top_left_x = x * n
    top_left_y = y * n
    board[top_left_x:top_left_x+n, top_left_y:top_left_y+n, :] = numpy.array(Image.open("Start1.png"))
    img = Image.fromarray(board.copy(), 'RGBA')
    images.append(img)
    
    count = 1
    while count < tile_size * n:
        next_x = x + directions[current_dir][0]
        next_y = y + directions[current_dir][1]
        if 0 <= next_x < tile_size and 0 <= next_y < tile_size and grid[next_x][next_y] == 0:
            x, y = next_x, next_y
            count += 1
            grid[x][y] = count
            
            top_left_x = x * n
            top_left_y = y * n
            predictions = model.predict(board[top_left_x-16:top_left_x+n+16, top_left_y-16:top_left_y+n+16, :].copy().reshape((-1, 48, 48, 4)))
            predictions = numpy.clip(predictions, 0, 255)
            predictions = numpy.where(predictions < 128, 0, 255)
            prediction = predictions.astype(numpy.uint8)[0]
            board[top_left_x:top_left_x+n, top_left_y:top_left_y+n, :] = prediction
            img = Image.fromarray(board.copy(), 'RGBA')
            images.append(img)
            
            steps_taken += 1
        
        if steps_taken == steps:
            current_dir = (current_dir + 1) % 4
            steps_taken = 0
            changes += 1
            if changes % 2 == 0:
                steps += 1

    return images

n = 16
tile_size = 48

frames = spiral_traverse_and_capture(tile_size, n, model=model)

frames[0].save('spiral_animation.gif',
            save_all=True,
            append_images=frames[1:],
            duration=100,
            loop=0)